In [1]:
from ultralytics import NAS

# Load a COCO-pretrained YOLO-NAS-s model
model = NAS("yolo_nas_s.pt")

# Display model information (optional)
model.info()

(521, 19053888, 19053888, 0.0)

In [5]:
import torch
from ultralytics import NAS

# Load pretrained YOLO-NAS small model (you can also use yolo_nas_m.pt or yolo_nas_l.pt)
model = NAS("yolo_nas_s.pt")

# Run inference on an image
results = model.predict(source="sample1.jpeg", save=True, conf=0.25)

# Loop through detections
for result in results:
    print("Image path:", result.path)
    print("Boxes:", result.boxes.xyxy)    # [x1, y1, x2, y2]
    print("Classes:", result.boxes.cls)   # class indices
    print("Confidences:", result.boxes.conf)  # confidence scores


In [2]:
pip install super-gradients opencv-python torch torchvision

In [3]:
import torch
import cv2
import numpy as np
from super_gradients.training import models
from super_gradients.common.object_names import Models

def load_yolo_nas_model(model_name='yolo_nas_l', device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Load pretrained YOLO-NAS model
    Available models: yolo_nas_s, yolo_nas_m, yolo_nas_l
    """
    model = models.get(model_name, pretrained_weights="coco").to(device)
    model.eval()
    return model

def predict_and_draw(image_path, model, confidence_threshold=0.5, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Perform object detection and draw bounding boxes on the image
    """
    # Read and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform prediction
    predictions = model.predict(image, conf=confidence_threshold)
    
    # Draw predictions on the image
    result = predictions._images_prediction_lst[0].draw()
    
    # Convert back to BGR for OpenCV display
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    
    # Display the result
    cv2.imshow('YOLO-NAS Detection', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return predictions

def print_detection_summary(predictions):
    """
    Print detection summary with class names and confidence scores
    """
    class_names = predictions._images_prediction_lst[0].class_names
    labels = predictions._images_prediction_lst[0].prediction.labels
    confidence_scores = predictions._images_prediction_lst[0].prediction.confidence
    bboxes = predictions._images_prediction_lst[0].prediction.bboxes_xyxy
    
    print("\nDetection Summary:")
    print(f"{'Class':<20} {'Confidence':<15} {'BBox Coordinates'}")
    print("-" * 60)
    for label, conf, bbox in zip(labels, confidence_scores, bboxes):
        class_name = class_names[int(label)]
        print(f"{class_name:<20} {conf:<15.2f} {bbox}")

def save_result(image_path, output_path, predictions):
    """
    Save the detection result to a file
    """
    result = predictions._images_prediction_lst[0].draw()
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, result)
    print(f"Result saved to {output_path}")

if __name__ == "__main__":
    # Configuration
    MODEL_NAME = 'yolo_nas_l'  # Options: yolo_nas_s, yolo_nas_m, yolo_nas_l
    IMAGE_PATH = 'sample.jpg'  # Replace with your image path
    OUTPUT_PATH = 'detection_result.jpg'
    CONFIDENCE_THRESHOLD = 0.5
    
    # Load model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    model = load_yolo_nas_model(MODEL_NAME, device)
    print(f"Loaded {MODEL_NAME} model successfully")
    
    # Perform detection
    predictions = predict_and_draw(IMAGE_PATH, model, CONFIDENCE_THRESHOLD, device)
    
    # Print detection summary
    print_detection_summary(predictions)
    
    # Save result
    save_result(IMAGE_PATH, OUTPUT_PATH, predictions)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [6]:

  import os
  from pathlib import Path
  import shutil

  from ultralytics.utils.downloads import download


  def visdrone2yolo(dir, split, source_name=None):
      """Convert VisDrone annotations to YOLO format with images/{split} and labels/{split} structure."""
      from PIL import Image
      from tqdm import tqdm

      source_dir = dir / (source_name or f"VisDrone2019-DET-{split}")
      images_dir = dir / "images" / split
      labels_dir = dir / "labels" / split
      labels_dir.mkdir(parents=True, exist_ok=True)

      # Move images to new structure
      if (source_images_dir := source_dir / "images").exists():
          images_dir.mkdir(parents=True, exist_ok=True)
          for img in source_images_dir.glob("*.jpg"):
              img.rename(images_dir / img.name)

      for f in tqdm((source_dir / "annotations").glob("*.txt"), desc=f"Converting {split}"):
          img_size = Image.open(images_dir / f.with_suffix(".jpg").name).size
          dw, dh = 1.0 / img_size[0], 1.0 / img_size[1]
          lines = []

          with open(f, encoding="utf-8") as file:
              for row in [x.split(",") for x in file.read().strip().splitlines()]:
                  if row[4] != "0":  # Skip ignored regions
                      x, y, w, h = map(int, row[:4])
                      cls = int(row[5]) - 1
                      # Convert to YOLO format
                      x_center, y_center = (x + w / 2) * dw, (y + h / 2) * dh
                      w_norm, h_norm = w * dw, h * dh
                      lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

          (labels_dir / f.name).write_text("".join(lines), encoding="utf-8")


  # Download (ignores test-challenge split)
  dir = Path(yaml["path"])  # dataset root dir
  urls = [
      "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip",
      "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip",
      "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip",
      # "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-challenge.zip",
  ]
  download(urls, dir=dir, threads=4)

  # Convert
  splits = {"VisDrone2019-DET-train": "train", "VisDrone2019-DET-val": "val", "VisDrone2019-DET-test-dev": "test"}
  for folder, split in splits.items():
      visdrone2yolo(dir, split, folder)  # convert VisDrone annotations to YOLO labels
      shutil.rmtree(dir / folder)  # cleanup original directory

NameError: name 'yaml' is not defined

In [21]:
from ultralytics import NAS
import cv2

# Load model
model = NAS("yolo_nas_s.pt")

# Run inference
results = model("sample.jpeg")

# Load original image
img = cv2.imread("sample.jpeg")

# Iterate over detections
for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())   # Bounding box
    conf = float(box.conf[0])                         # Confidence
    cls = int(box.cls[0])                             # Class index
    label = f"{model.names[cls]} {conf:.2f}"          # Class name + confidence

    # Draw rectangle and label
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Save output
cv2.imwrite("sample_detected.jpeg", img)


True